<a href="https://colab.research.google.com/github/ainfanzon/CRL-IAM-Workshop/blob/main/source/wrkshp_repl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.

## Identity and Access Management Workshop
---
## CockroachDB Overview

### Background Concepts

This section explains the concepts of ranges, range sets, replicas and leaders (a.k.a, leaseholders) which are the building blocks for
[Replication and Rebalancing](https://www.cockroachlabs.com/docs/stable/demo-replication-and-rebalancing).

In this section you will:

1. Start a 3-node cluster.
1. Check the status in the DB Console
1. Load some data and verify that it replicates in triplicate by default.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1roJY0K02x6gDV96uXT24ivR_f2gnf2Pb" width="350" height="345">
      </td>
      <td style="width:5%" align="center">
          &emsp;
      </td>
      <td align="left">
          <img src="https://drive.google.com/uc?id=19C2KDL00TdFwcZQX2epcqzMUi4evEMio" width="525" height="275">
      </td>
  </tr>
</table>

</body>
</html>

#### 1. Start a 3-node cluster.

Copy and paste the following statements to your terminal.

<code>
cockroach start --insecure --listen-addr=10.0.1.2:26257 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8080 --store=/home/roach_data/cr_data_1 --background<br>
cockroach start --insecure --listen-addr=10.0.1.2:26258 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8081 --store=/home/roach_data/cr_data_2 --background<br>
cockroach start --insecure --listen-addr=10.0.1.2:26259 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8082 --store=/home/roach_data/cr_data_3 --background<br>
</code>


<code>cockroach init --insecure --host 10.0.1.2</code>

You should see this `Cluster successfully initialized` message.

Use __`psycopg2`__ to connect to the database cluster using the *__VM External IP__*

In [7]:
import psycopg2
import pandas as pd

from IPython.display import IFrame, display, HTML, Markdown

conn = psycopg2.connect(
        database = 'defaultdb'
      , user = 'root'
      , host = '34.94.20.153'                        # Use the GCP Compute Engine external IP address
      , port = '26257'
      , sslmode = 'disable'
)
cursor = conn.cursor()

#### Validate the cluster has three nodes.

In [17]:
cursor.execute("""
SELECT max(node_id)
FROM crdb_internal.gossip_liveness
""")
res_tuple = cursor.fetchone()
display(Markdown(f'<h4><font color=green>CockroachDB cluster has: {res_tuple[0]} node(s)</font></h4>'))

<h4><font color=green>CockroachDB cluster has: 3 node(s)</font></h4>

In [23]:
cursor.execute("""
SELECT gn.node_id AS "Node ID"
     , gn.sql_address AS "SQL Address"
     , gn.advertise_sql_address AS "Advertised Address"
     , gn.build_tag AS "Version"
     , gn.started_at AS "Started At"
     , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
     , "ranges" AS "Ranges"
     , leases AS "Leaders"
     , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "status"
     , gl.membership
FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Node ID', inplace=True)
df_result_set

,SQL Address,Advertised Address,Version,Started At,Up Time,Ranges,Leaders,status,membership
Node ID,,,,,,,,,
1,10.0.1.2:26257,10.0.1.2:26257,v24.2.3,2024-10-18 16:49:36.659904,0 days 05:29:54.775072,55,20,LIVE,active
2,10.0.1.2:26259,10.0.1.2:26259,v24.2.3,2024-10-18 16:49:37.092819,0 days 05:29:54.342157,55,16,LIVE,active
3,10.0.1.2:26258,10.0.1.2:26258,v24.2.3,2024-10-18 16:49:37.417918,0 days 05:29:54.017058,55,19,LIVE,active


## Step 2. Load Data

In [ ]:
!/opt/homebrew/bin/cockroach workload init intro 'postgres://root@localhost:26257?sslmode=disable'
%sql cockroachdb://root@localhost:26257/system?sslmode=disable
mytable = %sql SELECT * FROM intro.mytable WHERE (l % 2) = 0
print(mytable)

In [ ]:
mytable = %sql SELECT * FROM intro.mytable WHERE (l % 2) = 0
print(mytable)

---
## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.
<img src="https://github.com/ainfanzon/Aerospike-Graph/blob/main/source/img/CockroachLabs_Logo.png?raw=1">

---